In [1]:
import numpy as np
import scipy.stats as stats
import itertools
from tqdm import tqdm

from pathlib import Path
import os
import sys
sys.path.append(os.path.dirname(Path.cwd()))
from src.frank_wolfe import (
    frank_wolfe, 
    select_top_k_np, 
    calculate_confusion_matrix_np, 
    macro_sqrt_tp_C,
    macro_jaccard_C, 
    macro_precision_C,
    macro_f1_C,
    macro_recall_C,
    precision_at_k_C,
)


np.set_printoptions(suppress=True)

# Definitions of some useful functions
def fw_output_to_sampling_proba(dist, classifiers, classifiers_weights, k):
    sampling_proba = []
    for i in range(dist.shape[0]):
        etas = dist[i]
        y = np.zeros((len(classifiers), len(etas)))
        for j in range(len(classifiers)):
            top_k = select_top_k_np(etas, classifiers[j], k)
            y[j,top_k] = classifiers_weights[j]
        y = y.sum(axis=0)
        sampling_proba.append(y)
    return np.array(sampling_proba)


def get_solution_using_fw(dist, k, utility_func):
    output = frank_wolfe(dist, 
                         dist, 
                         utility_func, 
                         max_iters=1000, 
                         k=k,
                         alpha_search_step = 0.0001, # step size for alpha search
                         use_best_alpha=False, # use best alpha for each iteration instead of the 2 / (i + 1) rule
                         verbose=False # print the utility value, classfiers and alpha at each iteration
                        )
    classifiers, classifiers_weights, meta = output
    sampling_proba = fw_output_to_sampling_proba(dist, classifiers, classifiers_weights, k=k)
    return sampling_proba


def evaluate_sampling_proba(sampling_proba, dist, utility_func):
    C = calculate_confusion_matrix_np(dist, sampling_proba, (len(dist[0]), 3))
    return utility_func(C).mean()
    

In [ ]:
# Two disitrubtions to compare
# eta of 1, 2, and 3 label for each instance
dist1 = [
    [0.4, 0.2, 0.6],
    [0.8, 0.4, 0.4]
]

dist2 = [
    [0.4, 0.2, 0.6],
    [0.8, 0.4, 0.8]
]

dist1 = np.array(dist1)
dist2 = np.array(dist2)

# in (macro_sqrt_tp_C, macro_jaccard_C, macro_precision_C, macro_f1_C, macro_recall_C, precision_at_k_C)
utility_func = macro_f1_C


sol1 = get_solution_using_fw(dist1, 2, utility_func)
sol2 = get_solution_using_fw(dist2, 2, utility_func)

print(f"solution 1:\n{sol1}\nutility: {evaluate_sampling_proba(sol1, dist1, utility_func)}")
print(f"solution 2:\n{sol2}\nutility: {evaluate_sampling_proba(sol2, dist2, utility_func)}")

In [ ]:
# Better solution for macro_f1_C (found using Bayseian optimization)
sol3 = np.array([[1, 0, 1], [0.76810359, 0.69829113, 0.53360528]])
print(f"solution 3:\n{sol3}\nutility: {evaluate_sampling_proba(sol3, dist2, macro_f1_C)}")
#0.6046468639156677

In [ ]:
from math import comb

# Generate all possible sampling distributions for a single instance (combinations) (BFS-style algo.)
def combinations_bfs(m, k, solution, step, array=[], sum=0):
    if len(array) > m or sum > k: 
        return
    if sum == k:
        solution.append(array + [0 for _ in range(m - len(array))])
        return
    for i in np.arange(0, 1 + step, step):
        combinations_bfs(m, k, solution, step, array + [i], sum + i)

def combinations(m, k, step=0.01):
    solution = []
    combinations_bfs(m, k, solution, step=step)
    return solution

def grid_search(dist, utility_func, k, step_size=0.01):
    sol = [(0, None)]
    sampling_prob_comb = combinations(len(dist[0]), k, step=step_size)
    for y_comb in tqdm(itertools.combinations_with_replacement(sampling_prob_comb, len(dist)), total=comb(len(sampling_prob_comb), len(dist))):
        for y_perm in itertools.permutations(y_comb):
            utility = evaluate_sampling_proba(np.array(y_perm), dist, utility_func)
            if utility > sol[0][0]:
                sol = [(utility, y_perm)]
                print(f"new best: {sol}")
            elif utility == sol[0][0]:
                sol.append((utility, y_perm))
                print(f"new best: {sol}")
    return sol

# This will take a few minutes to run
gs_sol1 = grid_search(dist1, utility_func, k=2)
gs_sol2 = grid_search(dist2, utility_func, k=2)
gs_sol1, gs_sol2

In [ ]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
k = 2
pbounds = {f'y{i}': (0.000001, 1) for i in range(6)}


def bo_sampling_proba(y0, y1, y2, y3, y4, y5):
    y = np.array([y0, y1, y2, y3, y4, y5])
    y = y.reshape((2, 3))
    if y[0].sum() < 2 or y[1].sum() < 2:
        return 0
    y[0] = y[0] / y[0].sum() * k
    y[1] = y[1] / y[1].sum() * k
    return y


def bo_utility_func(y0, y1, y2, y3, y4, y5):
    y = bo_sampling_proba(y0, y1, y2, y3, y4, y5)
    return evaluate_sampling_proba(y, dist2, utility_func)


optimizer = BayesianOptimization(f=bo_utility_func, pbounds=pbounds, random_state=1)

In [ ]:
# Rerun this function if you want it to do more iterations
optimizer.maximize(init_points=200, n_iter=500)

In [ ]:
max_params = max(optimizer.res, key=lambda x: x['target'])
sol = bo_sampling_proba(**max_params['params'])
sol, evaluate_sampling_proba(sol, dist2, utility_func)